In [37]:
import numpy as np
from scipy import io
#from scipy.stats import zscore
import matplotlib.pyplot as plt

In [20]:
def zscore(X, axis=0):
    mean_X= np.mean(X,axis=axis)
    std_X = np.std(X, axis=axis) + 1e-10
    X -= np.expand_dims(mean_X, axis)
    X /= np.expand_dims(std_X, axis)

    return X

In [48]:
class DataNode():
    def __init__(self, activity_patterns):
        self.n_neurons=activity_patterns.shape[0]
        self.activity_patterns=activity_patterns
        
class SimuNode():
    def __init__(self,n_neurons):
        self.n_neurons=n_neurons
        
class Synapse():
    def __init__(self,pre_synaptic_node, post_synaptic_node):
        self.pre_synaptic_node=pre_synaptic_node
        self.post_synaptic_node=post_synaptic_node
        
    def connect(self):
        row_dim=self.pre_synaptic_node.n_neurons
        col_dim=self.post_synaptic_node.n_neurons
        self.syn_weights=np.random.normal(loc=0,size=(col_dim,row_dim))
        
        
class CodeRunner():
    def __init__(self,connectivity_graph,node_classification):
        self.connectivity_graph=connectivity_graph
        self.node_classification=node_classification
    
    def gain_function(self,x):
        x[x<0]= 5*np.tanh(x[x<0]/5)
        '''
        if x<0:
            return 5*np.tanh(x/5)
        else:
            return (100-5)*np.tanh(x/(100-5))
        '''
        x[x>=0]=(100-5)*np.tanh(x[x>=0]/(100-5))
        return x
        
    def first_sim(self,data_path):
        activity_patterns=zscore(io.loadmat(data_path)['stim'][0]['resp'][0].T,axis=1)
        print(np.mean(activity_patterns,axis=1))
        print(activity_patterns.shape)
        dat=DataNode(activity_patterns)
        sim=SimuNode(2)
        syn=Synapse(dat,sim)
        syn.connect()
        print(syn.syn_weights.shape)
        synapse_lst=[syn]
        
        rate=np.array([[0,0]])
        h=0.01
        for t in range(0,1000):
            for syn in synapse_lst:
                inpt=syn.syn_weights@activity_patterns[:,t]
                dxdt=(-rate[-1]+self.gain_function(inpt))
                rate.vstack((rate[-1]+h*dxdt)[0])
                print(inpt)
        print(rate)
        plt.plot(rate)
        
    def second_sim(self,data_path):
        activity_patterns1=io.loadmat(data_path)['stim'][0]['resp'][0].T[:5000,:]
        activity_patterns2=io.loadmat(data_path)['stim'][0]['resp'][0].T[5000:,:]
        node_lst=[]
        for node in self.node_classification:
            if node=='simulated_inh':
                node_lst.append(SimuNode(1))
            if node=='simulated_exc':
                node_lst.append(SimuNode(1))
            if node=='data_inh':
                node_lst.append(DataNode(activity_patterns1))
            if node=='data_exc':
                node_lst.append(DataNode(activity_patterns2))
        synapse_lst=[]
        for row in range(0,self.connectivity_graph.shape[0]):
            for col in range(0,self.connectivity_graph.shape[1]):
                if self.connectivity_graph[row,col]==1:
                    syn=Synapse(node_lst[row],node_lst[col])
                    syn.connect()
                    synapse_lst.append(syn)
                    
        for syn in synapse_lst:
            print(syn.syn_weights.shape)  

In [46]:
path='/media/maria/DATA1/Documents/old_data/natimg2800_white_M170717_MP034_2017-09-14.mat'
run=CodeRunner(0,0).first_sim(path)

[-4.5323137e-08 -8.4980876e-09  4.5323137e-08 ... -5.2876992e-08
 -4.5323137e-08 -1.5107712e-08]
(9479, 4040)
(2, 9479)


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [30]:
node_classification=['data_exc','data_inh','simulated_exc']
connectivity_graph=np.array([[0,0,1],[0,0,1],[0,0,1]])
run=CodeRunner(connectivity_graph,node_classification).second_sim(path)

(1, 4479)
(1, 5000)
(1, 1)
